In [ ]:
from web3 import Web3
import json
web3 = Web3(Web3.HTTPProvider("http://localhost:8545"))
assert web3.isConnected()

In [ ]:
with open ("contract/abi.json") as f:
    contract_abi = f.read()
with open("config.json") as f:
    config = json.load(f)


In [ ]:
contract_address = config["contract_address"]
private_key = config["private_key1"]

In [ ]:
contract = web3.eth.contract(address=contract_address, abi=contract_abi)
account = web3.eth.account.from_key(private_key)
web3.eth.default_account = account.address
print("address : %s" % account.address)
nonce = web3.eth.getTransactionCount(account.address)

In [ ]:
tx_params = {
    'nonce': nonce,
    'gasPrice': web3.eth.gas_price,
    'gas': 5000000, 
}

In [ ]:
new_project_tx = contract.functions.new_project("project1").buildTransaction(tx_params)
signed_tx = web3.eth.account.sign_transaction(new_project_tx, private_key)
try:
    tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)
    tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print ("tx_receipt : %s" % tx_receipt)
except Exception as e:
    print (str(e).split("\'")[3])

In [ ]:
projects = contract.functions.get_all_projects_name().call()

for idx,p in enumerate(projects):
    print("%d. %s" % (idx+1,p))

In [ ]:
# cid = IPFS_API.Publish('./test.txt')
# print(cid)
# print(IPFS_API.CatFile(cid).decode('UTF-8', 'ignore'))